In [1]:
import requests
import json
import pandas as pd
import numpy as np

#### Create a playlist

In [2]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public = False):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

### Get a user's recently played songs
- `limit`: The maximum number of items to return. Default: `20`. Minimum: `1`. Maximum: `50`.
- `after`: A Unix timestamp in milliseconds. Returns all items after (but not including) this cursor position. If `after` is specified, `before` must not be specified.
- `before`: A Unix timestamp in milliseconds. Returns all items before (but not including) this cursor position. If `before` is specified, `after` must not be specified.

In [3]:
def get_recent_songs(user_id, token, **kwargs):
    query = "https://api.spotify.com/v1/me/player/recently-played"
    
    if len(kwargs.keys()) > 0:
        query += '?'
        lst = [str(x[0]) + '=' + str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all parameters from kwargs 
    #print(query)

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    print("status:", response.status_code)
    song_ids = [json_response['items'][i]['track']['id'] for i in range(len(json_response['items']))]
    return song_ids

### Get a User's Top Artists and Tracks
- `time_range`: Over what time frame the affinities are computed. Valid values: `long_term` (calculated from several years of data and including all new data as it becomes available), `medium_term` (approximately last 6 months), `short_term` (approximately last 4 weeks). Default: `medium_term`
- `limit`: The maximum number of items to return. Default: `20`. Minimum: `1`. Maximum: `50`.
- `offset`: The index of the first entity to return. Default: `0` (i.e., the first track). Use with limit to get the next set of entities.

**Note: songs that have been played for once will not be considered as a top song.**

In [4]:
def get_top_songs(user_id, token, **kwargs):
    query = "https://api.spotify.com/v1/me/top/tracks?"
    
    if len(kwargs.keys()) > 0:
        lst = [str(x[0]) + '=' + str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all parameters from kwargs 
    #print(query)

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    print("status:", response.status_code)
    song_ids = [json_response['items'][i]['id'] for i in range(len(json_response['items']))]
    return song_ids

### Get information about a song

- `market`: The market you’d like to request.

In [5]:
def get_song(user_id, token, song_id, market):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            df_response = pd.json_normalize(json_response)
            success = True
        except Exception as e:
            pass
            
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Accept": "application/json",
                                    "Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            album_id = json_response['album']['id'] if 'album' in json_response.keys() else None
            artist_id = json_response['artists'][0]['id'] if 'artists' in json_response.keys() else None
            success = True
        except Exception as e:
            pass
            
    if 'genre' in df_response.columns and df_response['genre'] is not None and len(df_response['genre']) > 0:
        return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')
    else:            
        ## get genre of the album
        base_url = "https://api.spotify.com/v1/albums/"
        query = f'{base_url}{album_id}?market={market}'
        success = False
        while not success:
            try:
                response = requests.get(query, 
                               headers={"Accept": "application/json",
                                        "Content-Type":"application/json", 
                                        "Authorization":f"Bearer {token}"})
                json_response = response.json()
                genre = json_response['genres'] if 'genres' in json_response.keys() else None
                success = True
            except Exception as e:
                pass

        if genre is None or len(genre) < 1:
            ## get genre of the artist
            base_url = "https://api.spotify.com/v1/artists/"
            query = f'{base_url}{artist_id}'
            success = False
            while not success:
                try:
                    response = requests.get(query, 
                                   headers={"Accept": "application/json",
                                            "Content-Type":"application/json", 
                                            "Authorization":f"Bearer {token}"})
                    json_response = response.json()
                    genre = json_response['genres'] if 'genres' in json_response.keys() else None
                    success = True
                except Exception as e:
                    pass

    df_response['artist_id'] = artist_id if artist_id is not None and len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if album_id is not None and len(album_id) >0 else None
    df_response['genre'] = [genre] if genre is not None and len(genre) > 0 else None
    try:
        return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')
    except Exception:
        print("Cannot find features about this song: ", song_id)

## Generate a target playlist

- `workout_plan`: Workout plan. For example, `workout_plan=[(0, 0.2), (8, 0.9), (20, 0.4)]` means that in the first 8 minutes, to increase energy from 0.2 to 0.9 and then decrease energy to 0.4 in the next 12 minutes.
- `history_args`: Parameters used to extract recently played songs. For example, `history_args = {'limit': 15, 'after': 28885117}`.
- `start_song`: The first song in the target playlist. You can input the id of a song.
- `end_song`: The last song in the target playlist. You can input the id of a song.
- `pars_weight`: When calculating the difference between songs, we use a weighted average of the difference. 
- `genre`: If it is not None, we will filter songs based on genre.
- `recently`: If it is True, we will use recently played songs; otherwise, we will use top songs.

**Note 1. if `recently=True`, `history_args` can accept "limit", "after" (or "before"); otherwise, `history_args` can accept "limit", "time_range", "offset".** 

**Note 2. The required scopes of token is different for "top" and "recently played" songs.**

In [6]:
def generate_target(user_id, token, market, workout_plan, history_args = {}, start_song = None, end_song = None,
                    pars_weight = {'energy': 0.6, 'duration_ms': 0.4},
                    genre = None, recently = True, MAX_songs = 1000, time_interval = 15):
    
    convert = lambda x: 60 * 1000 * x ## convert to ms
    reconvert = lambda x: x / 60 / 1000 ## convert to minute
    intersection = lambda lst1, lst2: [value for value in lst1 if value in lst2]
    
    def get_songs(my_offset):
        ## Note: for recently played songs, it cannot iterate to get more songs right now, it can be improved later.
        if recently:
            song_ids = get_recent_songs(user_id, token, **history_args)
        else:
            offset = max(my_offset, history_args.get('offset', 0))
            history_args['offset'] = offset
            song_ids = get_top_songs(user_id, token, **history_args)
        if len(song_ids) > 0:
            song_ids = set(song_ids) ## remove duplicate songs
            song_df = pd.concat([get_song(user_id, token, id, market) for id in song_ids])
            return song_df
        else:
            return None
    
    def filter_genre(song_df):
        if genre is not None:
            i = 0
            while i < song_df.shape[0]:
                if song_df.genre[i] is None or len(intersection(genre, song_df.genre[i])) < 1:
                    song_df = song_df.drop(index = song_df.index[i], axis = 0)
                else:
                    i += 1
        return song_df[pars_weight.keys()]
    
    my_offset = history_args.get('offset', 0)
    song_df = get_songs(my_offset)
    if song_df is None:
        print(" There is no avaliable top songs, please change `history_args`.")
    else:
        song_df = filter_genre(song_df)
    while song_df.duration_ms.sum() / 60000 < workout_plan[-1][0] and song_df.shape[0] < MAX_songs:
        ## check total duration, if the sum of the duration of songs is smaller than the duration of the work out plan, 
        ## get more songs
        my_offset += history_args.get('limit', 20)
        new_song = get_songs(my_offset)
        if new_song is None:
            break
        else:
            new_song = filter_genre(new_song)
            song_df = pd.concat([song_df, new_song])
        print(song_df.shape)
    print("final shape", song_df.shape)
    ###############
    ## remove duplicates
    #if workout_plan[0][1] == 'none':
    #    workout_plan = [workout_plan[i] for i in range(1, len(workout_plan)) 
    #                    if workout_plan[i][1] != workout_plan[i-1][1] and workout_plan[i][1] != 'none']
    #else:
    #    workout_plan = [workout_plan[0]] + [workout_plan[i] for i in range(1, len(workout_plan)) 
    #                       if workout_plan[i][1] != workout_plan[i-1][1] and workout_plan[i][1] != 'none']

    if song_df.shape[0] < 2:
        print(" There are no enough top songs, please change `history_args`.")
    else:
        songs = []
        total_durations = 0
        
        if isinstance(start_song, str) and start_song in song_df.index:
            song_df = song_df.drop(index = start_song, axis = 0)
        elif start_song is None:
            max_duration = workout_plan[0][0] * 60000
            if 
            df = song_df[song_df['duration_ms'] < max_duration]
            start_song = df.index[np.argmin((df['energy'] - workout_plan[0][1]).apply(abs))]
            song_df = song_df.drop(index = start_song, axis = 0)
        
        if isinstance(end_song, str) and end_song in song_df.index:
            song_df = song_df.drop(index = end_song, axis = 0)  
        elif end_song is None:
            end_song = song_df.index[np.argmin((song_df['energy'] - workout_plan[-1][1]).apply(abs))]
            song_df = song_df.drop(index = end_song, axis = 0)

        print('Start!')
        total_durations += get_song(user_id, token, start_song, market)['duration_ms'][0]
        print('Add start song:', start_song, total_durations/60000)
        songs.append(start_song)
        current_energy = get_song(user_id, token, start_song, market)['energy'][0]

        song_id = None
        for i in range(1, len(workout_plan)):
            next_position = convert(workout_plan[i][0])
            if i == len(workout_plan)-1:
                next_position = next_position - get_song(user_id, token, end_song, market)['duration_ms'][0]
            next_energy = workout_plan[i][1]
            trend = '+' if next_energy - current_energy > 0 else '-'
            
            while total_durations < next_position:
                df = song_df[song_df.energy > current_energy] if trend == '+' else song_df[song_df.energy < current_energy]
                ### add a filter based on duration
                target_duration = next_position - total_durations
                df = df[df['duration_ms'].between(target_duration - time_interval*1000, target_duration + time_interval*1000)]
                print(target_duration, target_duration - time_interval*1000, target_duration + time_interval*1000)
                if df.shape[0] < 1:
                    break
                if any(next_position - total_durations >= df.duration_ms):
                    energy_weights = [0.5, 0.5]
                else:
                    energy_weights = [0.1, 0.9]

                energy_weights = np.array(energy_weights) * pars_weight['energy']
                weight = list(energy_weights) + [pars_weight['duration_ms']]
                diff_current = df['energy'] - current_energy
                diff_next = df['energy'] - next_energy
                diff_duration = reconvert(df['duration_ms'] - (next_position - total_durations))
                diff = ((pd.DataFrame(np.c_[np.array(diff_current), np.array(diff_next), np.array(diff_duration)], index = df.index).apply(abs))*weight).sum(axis = 1).sort_values()
                song_id = diff.index[0]
                total_durations += get_song(user_id, token, song_id, market)['duration_ms'][0]
                songs.append(song_id)
                print('Add song:', song_id, total_durations/60000)
                current_energy = get_song(user_id, token, song_id, market)['energy'][0]
                song_df = song_df.drop(index = song_id, axis = 0)
        
        durations_with_last = total_durations+get_song(user_id, token, end_song, market)['duration_ms'][0]
        if song_id is not None:
            durations_without_last = durations_with_last - get_song(user_id, token, song_id, market)['duration_ms'][0]
            if abs(durations_without_last-convert(workout_plan[i][0])) < abs(durations_with_last-convert(workout_plan[i][0])):
                songs.pop(-1)
            songs.append(end_song)
            total_durations += get_song(user_id, token, end_song, market)['duration_ms'][0]
            print('Add end song:', end_song, total_durations/60000)
        return songs

# Test

In [7]:
# settings
user_id = "pbwppse1hilahmk43ls424ao4"

#### Test recently played songs without genre

In [8]:
token = "BQAulHsGTsUOBZDvajncv9Ua6Ke5bfigQtCpAIvOVJaX46K-7QK5UZeVWkwmmjIsZyuMZN9GG-HAHhGiB4zR-YKdEo8VLVw-DLPfpbQbqTrswumhUC43jdDUyJIer8UxvqKWZVsTj6wT-PHQNxjDd3w-6T1UJwsfoU7Ym9rCAPPvaF3XVFnebyUbxJq1by6djhk5HhzpIcgu-B5AWhJb9coCFr3LLzk1SrDpeFN6bXuMeRm8m5lLq9cXVQ"

In [9]:
market = 'US'
workout_plan = [(0, 0.2), (8, 0.9), (20, 0.4), (28, 0.7)]
pars_weight = {'energy': 0.8, 'duration_ms': 0.2}

history_args = {'limit': 15, 'after': 28885117} ## {} means using default values

In [10]:
songs = generate_target(user_id, token, market, workout_plan, history_args, pars_weight = pars_weight, recently = True)
songs

status: 200
final shape (10, 2)


ValueError: attempt to get argmin of an empty sequence

See the duration for each song and the total duration

In [ ]:
durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
durations / 60/ 1000, durations.sum() / 60/ 1000

See the energy for each song

In [ ]:
pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

Create a playlist with those songs

In [13]:
#create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
#            name='Recently no genre', description='', public=False)

#### Test recently played songs with genre

You can go to https://developer.spotify.com/console/get-available-genre-seeds/ to find all avaliable genres.

In [14]:
pars_weight = {'energy': 0.8, 'duration_ms': 0.2}

history_args = {'limit': 15, 'after': 28885117} ## {} means using default values

genre = ['new french touch']

In [15]:
songs = generate_target(user_id, token, market, workout_plan, history_args, 
                    pars_weight = pars_weight, genre = genre)
songs

status: 200
status: 200
Start!
Add song: 7dF4sXP1ISIharu35gQ86k 9.456616666666667
Add song: 6Jk7mNRofCpHStChx1EYOj 14.440166666666666


['6n0E5CPhtGPLW1544OlSTg',
 '7dF4sXP1ISIharu35gQ86k',
 '6Jk7mNRofCpHStChx1EYOj',
 '7bzks4LGpQUuPKBzJ6iQ7y']

In [16]:
durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
durations/ 60/ 1000, durations.sum() / 60/ 1000

(id
 6n0E5CPhtGPLW1544OlSTg    4.891550
 7dF4sXP1ISIharu35gQ86k    4.565067
 6Jk7mNRofCpHStChx1EYOj    4.983550
 7bzks4LGpQUuPKBzJ6iQ7y    4.573117
 Name: duration_ms, dtype: float64,
 19.013283333333334)

In [17]:
pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

id
6n0E5CPhtGPLW1544OlSTg    0.562
7dF4sXP1ISIharu35gQ86k    0.655
6Jk7mNRofCpHStChx1EYOj    0.646
7bzks4LGpQUuPKBzJ6iQ7y    0.700
Name: energy, dtype: float64

Create a playlist with those songs

In [18]:
#create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
#            name='Recently with a genre', description='', public=False)

#### Test top songs without genre

In [86]:
token = "BQAJqO5o5treOQyBGMOTaTEBcUGPDmogJbXitmBt7IhflfrTlGfVIkMd31kpTUmZXP97sCaFuU1upXOsFQ3H4FZA2FFhvGnk-vqdJhUMOJfiC_XCDbnIfuTP3EGHFRY1HboU_bo6pmn0yUwkNX0pytysWXF5p9qnMzzuNqMJ_VK7AwNlMWaDuVh_vm0LJYFhMWWkXT3jqehw2lpPFZiRrxqJki3EMDxEqOkwe03nTm2DkJsTTYWMYmeNQw"

In [87]:
pars_weight = {'energy': 0.8, 'duration_ms': 0.2}

history_args = {'limit': 15, 'time_range': 'long_term', 'offset': 0}

In [88]:
songs = generate_target(user_id, token, market, workout_plan, history_args, 
                    pars_weight = pars_weight, recently = False, genre = None)
songs

status: 200
status: 200
final shape (3, 2)


ValueError: attempt to get argmin of an empty sequence

In [79]:
durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
durations, durations.sum() / 60/ 1000

(id
 7nBR4Tt431p1MTgv3lVsmX    409558
 Name: duration_ms, dtype: int64,
 6.825966666666666)

In [80]:
pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

id
7nBR4Tt431p1MTgv3lVsmX    0.685
Name: energy, dtype: float64

Create a playlist with those songs

In [24]:
#create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
#            name='Top no genre', description='', public=False)

#### Test top songs with genre

In [25]:
pars_weight = {'energy': 0.8, 'duration_ms': 0.2}

history_args = {'limit': 15, 'time_range': 'medium_term', 'offset': 0}

genre = ['new french touch', 'trip hop', 'funky house']

In [26]:
songs = generate_target(user_id, token, market, workout_plan, history_args, 
                    pars_weight = pars_weight, recently = False, genre = genre)
songs

status: 200
status: 200
Start!
Add song: 09RXXnMSVrES6xju7IXrsX 13.234433333333333


['7nBR4Tt431p1MTgv3lVsmX', '09RXXnMSVrES6xju7IXrsX', '7bzks4LGpQUuPKBzJ6iQ7y']

In [27]:
durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
durations/ 60/ 1000, durations.sum() / 60/ 1000

(id
 7nBR4Tt431p1MTgv3lVsmX    6.825967
 09RXXnMSVrES6xju7IXrsX    6.408467
 7bzks4LGpQUuPKBzJ6iQ7y    4.573117
 Name: duration_ms, dtype: float64,
 17.80755)

In [28]:
pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

id
7nBR4Tt431p1MTgv3lVsmX    0.685
09RXXnMSVrES6xju7IXrsX    0.973
7bzks4LGpQUuPKBzJ6iQ7y    0.700
Name: energy, dtype: float64

Create a playlist with those songs

In [29]:
#create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
#            name='Top with a genre', description='', public=False)